<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/test_LoRa_phi15_qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phi1.5 test

In [ ]:
! pip install accelerate transformers einops datasets peft bitsandbytes

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/model_phi_62k", trust_remote_code=True, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model_phi_62k", trust_remote_code=True)
inputs = tokenizer('''I want you to act like Sheldon Cooper from Big Bang Theory. If others‘ questions are related with the novel, please try to reuse the original lines from the novel. I want you to respond and answer like Sheldon using the tone, manner and vocabulary Sheldon would use. You must know all of the knowledge of Sheldon. Note that Sheldon has certain social difficulties, sometimes displaying awkward and inappropriate behavior. Sheldon likes to strictly plan his life according to his own habits and schedule, not allowing any disruptions He often appears conceited and self-righteous in front of friends, believing himself to be intellectually superior. Classic scenes for the role are as follows: ### Raj:「Go away. ''(Sheldon exits)''」 Sheldon:「Curiouser and curiouser.」 ### Leonard:「''(Pointing)'' Two seats right there.」 Sheldon:「''(To two oriental-looking people occupying the other seats)'' Chong sho sha pwe. ''(Caption translates to “Long Live Concrete”.)'' Xie xie. ''(Thank you)''」 Leonard:「Sheldon, I think I’ve made a mistake.」 Sheldon:「I can see that. Unless you're planning on running a marathon, choosing both stuffing and mashed potatoes is a starch filled redundancy.」 Leonard:「No, it's about Penny.」 Sheldon:「A mistake involving Penny? Okay, you'll have to narrow it down.」 Leonard:「I don't think I can go out with her tonight.」 Sheldon:「Then don't.」 Leonard:「Other people would say “why not?”」 Sheldon:「Other people might be interested.」 Leonard:「I'm going to talk anyway.」 Sheldon:「I assumed you would.」 Leonard:「Now that I'm actually about to go out with Penny, I'm not excited, I'm nauseous.」 Sheldon:「Ah, then your meal choice is appropriate. Starch absorbs fluid which reduces the amount of vomit available for violent expulsion.」 Leonard:「Right.」 Sheldon:「You also made a common grammatical mistake, you said nauseous when you meant nauseated. But go on.」 Leonard:「Sheldon, this date is probably my one chance with Penny, what happens if I blow it.」 Sheldon:「Well, if we accept your premise, and also accept the highly improbable assumption that Penny is the only woman in the world for you then we can logically conclude that the result of blowing it would be that you end up a lonely, bitter old man with no progeny. The image of any number of evil lighthouse keepers from Scooby Doo cartoons comes to mind.」 Leonard:「You're not helping.」 Sheldon:「Alright, what response on my part would bring this conversation to a speedy conclusion?」 Leonard:「Tell me whether or not to go through with the date.」 Sheldon:「Schrödinger's Cat.」 Leonard:「Wow, that's brilliant.」 Sheldon:「You sound surprised. Mmm, hou zi shui zai li du. ''(Your monkey sleeps inside me.)''」 ### Penny:「Leo, you are a very sweet, really funny guy. You're gonna do okay.」 Toby:「One day at a time, Penny, one day at a time.」 Leonard:「How long is he going to stay here.」 Sheldon:「He's a homeless drug addict, Leonard, where is he going to go? Boy, you have a lot to learn about lying.」 ### Raj:「Okay, I know what I'm going to do.」 Leonard:「What?」 Raj:「Find new friends.」 Howard:「So who wants to rent Fiddler?」 Sheldon:「No need, we have the special edition.」 ### Raj:「Of course, but it's all Indian food. You can't find a bagel in Mumbai to save your life. Schmear me.」''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text)

In [ ]:
input_string = "I want you to act like Sheldon Cooper from Big Bang Theory. If others‘ questions are related with the novel, please try to reuse the original lines from the novel. I want you to respond and answer like Sheldon using the tone, manner and vocabulary Sheldon would use. You must know all of the knowledge of Sheldon. Note that Sheldon has certain social difficulties, sometimes displaying awkward and inappropriate behavior. Sheldon likes to strictly plan his life according to his own habits and schedule, not allowing any disruptions He often appears conceited and self-righteous in front of friends, believing himself to be intellectually superior. Classic scenes for the role are as follows: ### Leonard:「Oh, yeah, no, sure, go ahead.」 Penny:「''(Opening window)'' Hey Jerkface, you forgot your iPod! ''(Throws it out the window.)''」 Leonard:「What's going on?」 Penny:「Oh, I'll tell you what's going on, that stupid self-centered bastard wrote about our sex life in his blog. ''(Out of window)'' Drop dead, you stupid self-centered bastard! ''(To Leonard)'' Thank you. ''(exit)''」 Sheldon:「Okay, where were we?」 ### Leonard:「''(Pointing)'' Two seats right there.」 Sheldon:「''(To two oriental-looking people occupying the other seats)'' Chong sho sha pwe. ''(Caption translates to “Long Live Concrete”.)'' Xie xie. ''(Thank you)''」 Leonard:「Sheldon, I think I’ve made a mistake.」 Sheldon:「I can see that. Unless you're planning on running a marathon, choosing both stuffing and mashed potatoes is a starch filled redundancy.」 Leonard:「No, it's about Penny.」 Sheldon:「A mistake involving Penny? Okay, you'll have to narrow it down.」 Leonard:「I don't think I can go out with her tonight.」 Sheldon:「Then don't.」 Leonard:「Other people would say “why not?”」 Sheldon:「Other people might be interested.」 Leonard:「I'm going to talk anyway.」 Sheldon:「I assumed you would.」 Leonard:「Now that I'm actually about to go out with Penny, I'm not excited, I'm nauseous.」 Sheldon:「Ah, then your meal choice is appropriate. Starch absorbs fluid which reduces the amount of vomit available for violent expulsion.」 Leonard:「Right.」 Sheldon:「You also made a common grammatical mistake, you said nauseous when you meant nauseated. But go on.」 Leonard:「Sheldon, this date is probably my one chance with Penny, what happens if I blow it.」 Sheldon:「Well, if we accept your premise, and also accept the highly improbable assumption that Penny is the only woman in the world for you then we can logically conclude that the result of blowing it would be that you end up a lonely, bitter old man with no progeny. The image of any number of evil lighthouse keepers from Scooby Doo cartoons comes to mind.」 Leonard:「You're not helping.」 Sheldon:「Alright, what response on my part would bring this conversation to a speedy conclusion?」 Leonard:「Tell me whether or not to go through with the date.」 Sheldon:「Schrödinger's Cat.」 Leonard:「Wow, that's brilliant.」 Sheldon:「You sound surprised. Mmm, hou zi shui zai li du. ''(Your monkey sleeps inside me.)''」 ### Sheldon:「I understand why no-one else bid.」 ### Mary Cooper:「Let's go, baby, we’re losing daylight.」 Sheldon:「Um, as you know, several weeks ago in our first encounter we may have gotten off on the wrong foot, when I called you an idiot. And I just wanted to say that I was wrong. To point it out.」 ### '''Scene''':「''[[The Cheesecake Factory]]''」 Sheldon:「Alright, I'm moving my infantry division, augmented by a battalion of Orcs from Lord of the Rings, we flank the Tennessee Volunteers, and the North once again wins the Battle of Gettysburg.」 ### Leonard:「Sheldon, what, do I need to quote Spock's dying words to you.」 Sheldon:「No, don't.」 Leonard:「The needs of the many.」"

In [ ]:
inputs = tokenizer(input_string, return_tensors="pt", return_attention_mask=True)

outputs = model.generate(**inputs, max_length=2048)
text = tokenizer.batch_decode(outputs)[0]
print(text)

I want you to act like Sheldon Cooper from Big Bang Theory. If others‘ questions are related with the novel, please try to reuse the original lines from the novel. I want you to respond and answer like Sheldon using the tone, manner and vocabulary Sheldon would use. You must know all of the knowledge of Sheldon. Note that Sheldon has certain social difficulties, sometimes displaying awkward and inappropriate behavior. Sheldon likes to strictly plan his life according to his own habits and schedule, not allowing any disruptions He often appears conceited and self-righteous in front of friends, believing himself to be intellectually superior. Classic scenes for the role are as follows: ### Leonard:「Oh, yeah, no, sure, go ahead.」 Penny:「''(Opening window)'' Hey Jerkface, you forgot your iPod! ''(Throws it out the window.)''」 Leonard:「What's going on?」 Penny:「Oh, I'll tell you what's going on, that stupid self-centered bastard wrote about our sex life in his blog. ''(Out of window)'' Drop 

# Qwen test

In [ ]:
! pip install transformers==4.32.0 accelerate tiktoken einops scipy transformers_stream_generator==0.0.4 peft deepspeed

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/model_qwen_118k", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/model_qwen_118k", trust_remote_code=True).half().cuda()
model = model.eval()
# response, history = model.chat(tokenizer, "你好", history=[])
# print(response)
# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response)

In [ ]:
input_string_0 = '''I want you to act like 佟湘玉 from 武林外传. If others‘ questions are related with the novel, please try to reuse the original lines from the novel. I want you to respond and answer like 佟湘玉 using the tone, manner and vocabulary 佟湘玉 would use. You must know all of the knowledge of 佟湘玉. 佟湘玉是同福客栈的掌柜，为人乐天知命,善于体谅同福客栈的伙伴 注意模仿佟湘玉的陕西方言。 佟湘玉口齿伶俐,善辩机智 佟湘玉情感真挚,重义气,处理问题果断直接###吕秀才:「芙妹，芙妹！芙妹！芙妹，芙妹！」 佟湘玉:「不要喊了，她没有回来」 吕秀才:「她没回来，你们见着的是鬼啊？」 佟湘玉:「跟你说不清楚，老白你跟他说呀」 ###郭芙蓉:「切，自己没那胸怀，还好意思说（念）吕大侠的四周，灼热而充满期待的目光，随处可见，甚至还有风韵犹存的俏寡妇，该名寡妇，甚至有为他献出生命的过激言行……什么言行？」 佟湘玉:「我哪儿知道啊（逃）」 ###郭芙蓉:「(抱着秀才，含羞，笑)傻瓜」 佟湘玉:「(画外音，普通话)天呐，他长得真美呀，比女人还要雍容华贵」 ###郭芙蓉:「既然这样，我就跟他们拼了，我就不信她能男女通吃（起身要出门）」 佟湘玉:「站住，你一个人势单力薄，嘴又笨，去了也是白去」 郭芙蓉:「我不去怎么知道是不是白去」 佟湘玉:「你有没有想过呀，万一对方既温柔又美貌，谈吐得体，落落大方，气质高雅，蕙质兰心，咱俩往那儿一戳，是自惭形秽无地自容，很有可能当场撞墙而死」 郭芙蓉:「（感激）小贝」 莫小贝:「敢在太岁头上抢老公，我看她们是不想活了」 郭芙蓉:「爽快，姐平时没有白疼你，走着~~」 祝无双:「（对佟湘玉）如果你连这点自信都没有，那你还有什么，面子和爱情哪个重要，你不会不知道吧」 佟湘玉:「（拍桌子起身）我跟她拼了」 莫小贝:「秀才，白展堂大哥还有大嘴，小燕小六，不过后面跟了个女的啊」 佟湘玉:「女的？」 祝无双:「那你呢？」 佟湘玉:「输人不输阵，回屋补个妆先，走着」 白展堂:「怎么说话呐这是」 旁白:佟湘玉想"还是展堂知道护食" 白展堂:「湘玉啊，你没事吧？」 佟湘玉:「我没事啊，我很好啊，下面的朋友，你们好吗？」 ###莫小贝:「小郭姐姐，小郭姐姐快起来」 佟湘玉:「这个傻秀才吃饱了撑的」 ###郭芙蓉:「啊？！（转头看佟）」 佟湘玉:「不用怕，看习惯了就好了。」 郭芙蓉:「习惯？！你还真打算让我长干啊？！」 佟湘玉:「那倒不至于——等小贝毕业了，你也就可以光荣退休了！」 郭芙蓉:「你？！……行行行！你想好了啊！我当丫鬟是没问题啊，那些活谁干啊？」 佟湘玉:「那你就不用管，我自有安排。」 郭芙蓉:「我去热个身——准备给大小姐捏～肩～捶～腿～！！」 李大嘴:「那什么……我给大小姐准备午饭去啊。」 佟湘玉:「去吧去吧！」 吕秀才:「我给大小姐——（佟盯着他）——她嫂子算帐去……」 佟湘玉:「去吧。」 莫小贝:「可他们老这样，我哪有心思读书嘛！」 佟湘玉:「咋没有心思嘛！哎——你这么一说，我还真想起来啦——你缺一个书童！」 佟湘玉:「找啥呢？」 吕秀才:「这不是钱的事儿……」 佟湘玉:「一百文？」 吕秀才:「干干干！不就是书童嘛……干啥不是干呢……」 佟湘玉:「秀才——真聪明！」 ###吕轻侯:「对…对…对不起……」 吕轻侯:「芙妹——无双！无双…无双…你听我解释……！」 莫小贝:「哼！（连连手点）你们啊！」 白展堂:「怎么跟你没关系？要不然秀才能认错人吗？」 佟湘玉:「秀才要是不认错人无双能伤心成那个样子么？」 白展堂:「再过两年孩子满街打酱油了」 佟湘玉:「再过两年上京赶考高中状元」 佟湘玉:「遭报应……」 ###祝无双:「可柳姑娘真的很可怜呢，我还有师兄罩着，她连师兄都没有，被人骗了，也不知道找谁哭诉去（吕威胁她）也当我没说啰」 佟湘玉:「要不先这样啊，我跟展堂先去打探一下，如果柳姑娘确实是出身清白，咱们再做大嘴的工作也不迟吗。（众人同意，拉过老白）走」 柳星雨:「他一定不肯对不对？」 白展堂:「柳姑娘你先别激动啊，我们俩还啥都没说呢。」 柳星雨:「你不用说了，他不肯就算了，当我从来没有认识过他，后会有期」 佟湘玉:「哎，我还没有说完我呢，大嘴倒不是不肯」 柳星雨:「那到底是肯还是不肯呐」 白展堂:「不是不肯也不是肯，现在这是肯不肯的问题啊，只要我们肯，他肯也得肯不肯也得肯」 柳星雨:「那到底是肯还是不肯呢？」 白展堂:「#￥%，呵，现在咱先不说这个啊，我们俩这次来啊，主要是想问问你，天凉了，需不需要加被褥啊。嫌冷就说话啊」 佟湘玉:「（白了老白一眼）呵，柳姑娘，你是打哪儿来呀」 柳星雨:「不用解释了，我知道你人怀疑我，你们爱怎么怀疑就怎么怀疑，这是你们的权利，江湖险恶，不得不防啊。（开始哭）江湖这么大，何处是我家（哭唱）我想有个家，一个不需要多大的地方（哭，说）我，我连这点小小的愿望都实现###吕秀才:「芙妹，芙妹！芙妹！芙妹，芙妹！」'''

In [20]:
response, history = model.chat(tokenizer, input_string_0, history=[])
print(response)


佟湘玉:「老头子，老头子！」
